In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchmetrics

from nn import ANN
from eval import KFoldCV

In [ ]:
X_df = pd.read_csv('trn_X.csv', index_col='Id')
y_df = pd.read_csv('trn_y.csv', index_col='Id')

X, y = torch.tensor(X_df.to_numpy(dtype=np.float32)), torch.tensor(y_df.to_numpy(dtype=np.float32))

In [ ]:
model_kwargs = {
  'input_dim': X.shape[-1],
  'hidden_dim': 128,
  'dropout': 0.3,
}

dl_kwargs = {
  'batch_size': 36,
  'shuffle': True,
}

optim_kwargs = {
  'lr': 1.0,
}

device = 'cpu'

cv = KFoldCV(X, y, ANN, model_kwargs=model_kwargs,
            epochs=300,
            criterion=F.mse_loss,
            Optimizer=torch.optim.Adam,
            optim_kwargs=optim_kwargs,
            trn_dl_kwargs=dl_kwargs, val_dl_kwargs=dl_kwargs,
            metric=torchmetrics.MeanSquaredError(squared=False).to(device),
            device=device)


In [ ]:
res = cv.run()
res = pd.concat([res, res.apply(['mean', 'std'])])

res